In [1]:
import json

import folium
import numpy as np
import pandas as pd

from scipy.interpolate import interp1d

In [46]:
# Se sacan los del 13 y 14 porque no tienen bloque

df12 = pd.read_csv('viajes_2012.csv')
#df13 = pd.read_csv('viajes_2013.csv')
#df14 = pd.read_csv('viajes_2014.csv')
df15 = pd.read_csv('viajes_2015.csv')
df16 = pd.read_csv('viajes_2016.csv')
df17 = pd.read_csv('viajes_2017.csv')

colores = {'PJ Frente para la Victoria': '#a6cee3', 'Unión Cívica Radical':'#1f78b4',
'PJ La Pampa':'#b2df8a', 'Justicialista San Luis':'#33a02c',
 'Movimiento Popular Neuquino':'#fb9a99', 'Frente Cívico por Santiago':'#e31a1c',
 'Partido Socialista':'#fdbf6f', 'otros':'#ff7f00'}

In [40]:
# Unimos los df
df = pd.concat([df12, df15, df16, df17])

In [27]:
def check_dato(destino, bloque, list_datos, origen='Argentina'):
    for dato in list_datos:
        if destino == dato['destino'] and bloque == dato['bloque'] and origen == dato['origen']:
            dato['count'] = dato['count'] + 1
            return True
    if not '/' in destino:
        viaje = {'destino':destino, 'bloque':bloque, 'origen':origen, 'count':1}
        list_datos.append(viaje)
    else:
        # Atajo los dos casos
        if ' / ' in destino:
            escala, destino = destino.split(' / ', maxsplit=1)
        elif '/' in destino:
            escala, destino = destino.split('/', maxsplit=1)
        check_dato(destino, bloque, list_datos, escala)
    return True

datos = []
for i in df.itertuples():
    check_dato(i.Destino, i.Bloque, datos)
    
count_viajes_x_destino_bloque = pd.DataFrame(datos)

In [28]:
count_viajes_x_destino_bloque.destino.unique()

array(['Chile', 'México', 'Costa Rica', 'Francia', 'Italia', 'Uganda',
       'Estados Unidos', 'Uruguay', 'Colombia', 'Alemania', 'Ecuador',
       'Turquía', 'Panamá', 'Suiza', 'Holanda', 'Perú', 'Brasil',
       'República Dominicana', 'Israel', 'Canadá', 'Venezuela', 'España',
       'Cuba', 'Suecia', 'Grecia', 'Bolivia', 'Reino Unido', 'Corea',
       'Armenia', 'Puerto Rico', 'Bélgica', 'Letonia', 'Sudáfrica',
       'Rusia', 'Paraguay', 'Reino Unido de Gran Bretaña',
       'Guyana Francesa', 'República Checa', 'Portugal', 'Aruba',
       'Irlanda', 'Emiratos Árabes Unidos', 'Reino Unido de Gran  Bretaña',
       ' Irlanda del Norte', 'Estados Unidos de América',
       'República de Azerbaiyán', 'Bangladesh'], dtype=object)

In [29]:
df_counts = df.groupby('Destino')['Destino'].count()

countries_destino = set([country for country in count_viajes_x_destino_bloque.destino])
countries_origen = set([country for country in count_viajes_x_destino_bloque.origen])
countries = countries_destino.union(countries_origen)
# Elimino algos repetidos (con un espacil al principo)
list_countries = list(countries)

In [30]:
# Lat lon del centro de cada pais
with open('centro_paises.geojson', 'r') as file:
    data = json.load(file)

In [31]:
countries = dict()
for feature in data['features']:
    nombre = feature['properties']['nombre']
    if nombre in list_countries:
        countries[nombre] = {'latlon': [feature['geometry']['coordinates'][1],
                                        feature['geometry']['coordinates'][0]]}
        
countries

{'Alemania': {'latlon': [51.0871017663336, 10.370787457775123]},
 'Argentina': {'latlon': [-35.18209766247696, -65.1548248323]},
 'Armenia': {'latlon': [40.28859678761182, 44.9317308780777]},
 'Bangladesh': {'latlon': [23.88242325782727, 90.22663495543901]},
 'Bolivia': {'latlon': [-16.706876766549385, -64.68475359110516]},
 'Brasil': {'latlon': [-16.452174485463956, -47.50483868565664]},
 'Bélgica': {'latlon': [50.83668469454318, 4.370396733222833]},
 'Canadá': {'latlon': [59.65683609990623, -117.09059449134253]},
 'Chile': {'latlon': [-35.44787283276486, -71.24781565426889]},
 'Colombia': {'latlon': [3.911188855381511, -73.07834068147713]},
 'Costa Rica': {'latlon': [9.974655552412003, -84.19202644686288]},
 'Cuba': {'latlon': [22.093377645162377, -79.54124915322569]},
 'Ecuador': {'latlon': [-1.448279428452679, -78.39002628888198]},
 'Emiratos Árabes Unidos': {'latlon': [23.43367363652925, 53.97651207707088]},
 'España': {'latlon': [40.40029669653546, -3.566448279164604]},
 'Estados

In [62]:
countries['Argentina']['latlon'], countries['Alemania']['latlon']

([-35.18209766247696, -65.1548248323], [51.0871017663336, 10.370787457775123])

In [ ]:
atribution = ('Map data &copy; <a href="http://openstreetmap.org">OpenStreetMap</a> '
              'contributors, <a href="http://creativecommons.org/licenses/by-sa/2.0/">'
              'CC-BY-SA</a>, Imagery © <a href="http://mapbox.com">Mapbox</a>')

my_map = folium.Map(location=[45.372, -121.6972],
                   zoom_start=2,
                   tiles='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
                   attr=atribution)

# Countries markers
for country in list_countries:
    # Marker
    try:
        text = "{0}".format(country)
        folium.Marker(countries[country]['latlon'], 
                  popup=text).add_to(my_map)
    except:
        pass
# Add Argentina marker
#folium.Marker(origin, popup='Argentina').add_to(my_map)

# Connection lines
for i in count_viajes_x_destino_bloque.itertuples():
    try:
        lat = np.linspace(countries[i.origen]['latlon'][0], countries[i.destino]['latlon'][0], 3)
        lon = np.linspace(countries[i.origen]['latlon'][1], countries[i.destino]['latlon'][1], 3)

        # los paises que esta muy juntos los conecto por una recta
        if lat.max() - lat.min() > 5:
            lat[1] = lat[1] + np.random.choice([-1, 1])*(lat.max() - lat.min())/np.random.choice([4, 5])
            lon[1] = lon[1] + np.random.choice([-1, 1])*np.random.random()*15
            f2 = interp1d(lat, lon, kind='quadratic')
            #
            xnew = np.linspace(lat.min(), lat.max(), num=41, endpoint=True)
            coordinates = [[x, y] for x, y in zip(xnew, f2(xnew))]
        else:
            coordinates = [[x, y] for x, y in zip(lat, lon)]

        # Asignamos los colores si estan en los principales bloques
        if i.bloque in colores.keys():
            color = colores[i.bloque]
        else:
            color = colores['otros']

        folium.PolyLine(coordinates, weight=i.count, color=color).add_to(my_map)
    except:
        pass
my_map